In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import json
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


%matplotlib inline

In [2]:
path = '/Volumes/GoogleDrive/My Drive/wish_local/DSF-1436 Fusion Incrementality Analysis/'

In [3]:
df = pd.read_csv(path+'data/dsf1436_dau_gmv_price_by_month.csv', parse_dates=['month'])
df = df.sort_values(['month','bucket'])
df.head()

,month,bucket,dusers0,dusers,order_users,order_nusers,avg_orders,std_orders,avg_price_rev,std_price_rev,avg_fusion_orders,avg_gmv,std_gmv,orders_2,orders_5,orders_10,orders_20,orders_30,orders_50,fusion_orders_2,fusion_orders_5,fusion_orders_10,fusion_orders_20,fusion_orders_30,fusion_orders_50,time
9,2021-04-01,android_control,6328013,6328013,370602,16902,0.152562,0.954174,0.772170,5.535331,0.001869,1.965292,14.363650,89253,552661,727702,861811,920686,952731,982,7346,9492,10967,11524,11787,1629305993
15,2021-04-01,android_show,120644425,120644425,7075475,319583,0.153447,0.957688,0.746086,5.383284,0.018322,1.943394,14.310703,1656607,10625178,13984075,16544598,17665129,18274005,198103,1438739,1818393,2065242,2162068,2204441,1629305993
2,2021-04-01,ios_control,3565405,3565405,173515,9200,0.126580,0.852301,0.656926,5.051332,0.002963,1.679275,13.263878,39275,252168,336827,401244,429482,445137,914,6646,8581,9831,10289,10522,1629305993
7,2021-04-01,ios_show,67405117,67405117,3277838,172868,0.127761,0.907356,0.639251,5.394725,0.013924,1.663446,13.914742,737344,4859384,6456928,7665753,8198000,8497026,78206,589128,756825,869859,914797,935612,1629305993
4,2021-05-01,android_control,6314464,6314464,345614,14736,0.138125,0.878435,0.719712,5.314697,0.002191,1.793717,14.102868,83942,495192,658295,780687,833458,861368,1171,8552,11047,12716,13440,13786,1629305993


In [14]:
df['bucket'] = df['bucket'].apply(lambda x: x.upper())

In [42]:
df['orders'] = df['dusers'] * df['avg_orders']

# Price revenue

In [5]:
pr_cols = ['avg_price_rev',
        'std_price_rev']

In [25]:
pr = df[['month', 'bucket', 'dusers']+pr_cols].copy(deep=True)
pr

,month,bucket,dusers,avg_price_rev,std_price_rev
9,2021-04-01,ANDROID_CONTROL,6328013,0.772170,5.535331
15,2021-04-01,ANDROID_SHOW,120644425,0.746086,5.383284
2,2021-04-01,IOS_CONTROL,3565405,0.656926,5.051332
7,2021-04-01,IOS_SHOW,67405117,0.639251,5.394725
4,2021-05-01,ANDROID_CONTROL,6314464,0.719712,5.314697
8,2021-05-01,ANDROID_SHOW,120668163,0.692993,5.235179
1,2021-05-01,IOS_CONTROL,3521527,0.648401,5.124471
0,2021-05-01,IOS_SHOW,66655314,0.628853,5.983365
14,2021-06-01,ANDROID_CONTROL,5366595,0.668225,5.271721
13,2021-06-01,ANDROID_SHOW,102923157,0.641304,5.206875


## calculate diff


In [28]:
def show_control_diff(x, data, c, diff_pct=False):
    """
    x: row
    data: original dataframe
    c: column to calculate diff
    diff_pct: False is return absoluate diff, True is to return diff %
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        
        control_value = data[(data['month']==x['month']) & \
                             (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][c].iloc[0]
        
        show_value = x[c]
    
        if diff_pct:
            return (show_value - control_value) / control_value
        else:
            return show_value - control_value

In [29]:
for c in pr_cols:
    pr[c+'_diff'] = pr.apply(lambda x: show_control_diff(x, pr, c, diff_pct=True), axis=1)
    
temp = zip(['avg_price_rev'],\
           [c+'_diff' for c in ['avg_price_rev']])
temp_cols = [l1 for l2 in temp for l1 in l2]

pr[['month', 'bucket', 'dusers']+temp_cols].head()

,month,bucket,dusers,avg_price_rev,avg_price_rev_diff
9,2021-04-01,ANDROID_CONTROL,6328013,0.772170,NaN
15,2021-04-01,ANDROID_SHOW,120644425,0.746086,-0.033780
2,2021-04-01,IOS_CONTROL,3565405,0.656926,NaN
7,2021-04-01,IOS_SHOW,67405117,0.639251,-0.026905
4,2021-05-01,ANDROID_CONTROL,6314464,0.719712,NaN


## t-test

In [33]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# https://github.com/scipy/scipy/blob/v1.7.1/scipy/stats/stats.py#L5879-

from scipy.stats import distributions

def _unequal_var_ttest_denom(v1, n1, v2, n2):
    vn1 = v1 / n1
    vn2 = v2 / n2
    with np.errstate(divide='ignore', invalid='ignore'):
        dof = (vn1 + vn2)**2 / (vn1**2 / (n1 - 1) + vn2**2 / (n2 - 1))

    # If dof is undefined, variances are zero (assumes n1 > 0 & n2 > 0).
    # Hence it doesn't matter what dof is as long as it's not NaN.
    dof = np.where(np.isnan(dof), 1, dof)
    denom = np.sqrt(vn1 + vn2)
    return dof, denom

def _ttest_finish(dof, t, alternative):
    """Common code between all 3 t-test functions."""
    if alternative == 'less':
        prob = distributions.t.cdf(t, dof)
    elif alternative == 'greater':
        prob = distributions.t.sf(t, dof)
    elif alternative == 'two-sided':
        prob = 2 * distributions.t.sf(np.abs(t), dof)
    else:
        raise ValueError("alternative must be "
                         "'less', 'greater' or 'two-sided'")
    return t, prob

def _ttest_ind_from_stats(mean1, mean2, denom, dof, alternative):

    d = mean1 - mean2
    with np.errstate(divide='ignore', invalid='ignore'):
        t = np.divide(d, denom)
    t, prob = _ttest_finish(dof, t, alternative)

    return (t, prob)



def unequal_var_ttest(v1, v2, n1, n2, m1, m2, alternative='two-sided'):
    dof, denom = _unequal_var_ttest_denom(v1, n1, v2, n2)
    t, pval = _ttest_ind_from_stats(m1, m2, denom, dof, alternative)
    
    return t, pval

In [34]:
def show_control_t_test(x, data, avg_col, std_col, n_col, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        v_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][std_col].iloc[0] ** 2
        v_show = x[std_col] ** 2
        
        n_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][n_col].iloc[0]
        n_show = x[n_col]
        
        m_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][avg_col].iloc[0]
        m_show = x[avg_col]
        
        t, pval = unequal_var_ttest(v_control, v_show, n_control, n_show, m_control, m_show, alternative)
        
        return round(pval, 4)


In [36]:
for c in ['price_rev']:
    pr[c+'_pval'] = pr.apply(lambda x: \
                                 show_control_t_test(x, data=pr, \
                                                     avg_col='avg_'+c, \
                                                     std_col='std_'+c, \
                                                     n_col='dusers', \
                                                     alternative='two-sided') , axis=1)

    pr['avg_'+c+'_diff2'] = pr.apply(lambda x: f"{round(x['avg_'+c+'_diff']*100, 2)}%" + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip(['avg_price_rev'], \
           [c+'_diff2' for c in ['avg_price_rev']])#, [c+'_test' for c in pr_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

pr[['month', 'bucket', 'dusers']+temp_cols]

,month,bucket,dusers,avg_price_rev,avg_price_rev_diff2
9,2021-04-01,ANDROID_CONTROL,6328013,0.772170,nan%
15,2021-04-01,ANDROID_SHOW,120644425,0.746086,-3.38%***
2,2021-04-01,IOS_CONTROL,3565405,0.656926,nan%
7,2021-04-01,IOS_SHOW,67405117,0.639251,-2.69%***
4,2021-05-01,ANDROID_CONTROL,6314464,0.719712,nan%
8,2021-05-01,ANDROID_SHOW,120668163,0.692993,-3.71%***
1,2021-05-01,IOS_CONTROL,3521527,0.648401,nan%
0,2021-05-01,IOS_SHOW,66655314,0.628853,-3.01%***
14,2021-06-01,ANDROID_CONTROL,5366595,0.668225,nan%
13,2021-06-01,ANDROID_SHOW,102923157,0.641304,-4.03%***


In [38]:
def color_pval(x):
    if '*' in str(x):
        if '-' in str(x):
            return "background-color: pink"
        else:
            return "background-color: lightgreen"
    return "background-color: white"

In [39]:
pr[['month', 'bucket', 'dusers']+temp_cols].reset_index().style.applymap(color_pval)

,index,month,bucket,dusers,avg_price_rev,avg_price_rev_diff2
0,9,2021-04-01 00:00:00,ANDROID_CONTROL,6328013,0.772170,nan%
1,15,2021-04-01 00:00:00,ANDROID_SHOW,120644425,0.746086,-3.38%***
2,2,2021-04-01 00:00:00,IOS_CONTROL,3565405,0.656926,nan%
3,7,2021-04-01 00:00:00,IOS_SHOW,67405117,0.639251,-2.69%***
4,4,2021-05-01 00:00:00,ANDROID_CONTROL,6314464,0.719712,nan%
5,8,2021-05-01 00:00:00,ANDROID_SHOW,120668163,0.692993,-3.71%***
6,1,2021-05-01 00:00:00,IOS_CONTROL,3521527,0.648401,nan%
7,0,2021-05-01 00:00:00,IOS_SHOW,66655314,0.628853,-3.01%***
8,14,2021-06-01 00:00:00,ANDROID_CONTROL,5366595,0.668225,nan%
9,13,2021-06-01 00:00:00,ANDROID_SHOW,102923157,0.641304,-4.03%***


# Price category
do the lower priced goods sell more?
1. price category: b.merchant_price + b_merchant_shipping, 0-2, 2-5, 5-10, 10-20, 20-30, 30-50, 50+
2. metric 1: in show bucket, is the low-price-category orders / all orders < low-price-category fusion orders / all fusion orders?
3. metric 2: is low-price-category orders / all orders in show bucket < low-price-category orders / all orders in control bucket?

In [59]:
df[['month', 'bucket', 'dusers']+['orders', 'orders_2']].head()

,month,bucket,dusers,orders,orders_2
9,2021-04-01,ANDROID_CONTROL,6328013,965417.0,89253
15,2021-04-01,ANDROID_SHOW,120644425,18512530.0,1656607
2,2021-04-01,IOS_CONTROL,3565405,451310.0,39275
7,2021-04-01,IOS_SHOW,67405117,8611742.0,737344
4,2021-05-01,ANDROID_CONTROL,6314464,872188.0,83942


In [60]:
prop = df[['month', 'bucket', 'dusers', \
           'orders', 'orders_2', 'orders_5', 'orders_10', 'orders_20', 'orders_30', 'orders_50']].copy(deep=True)

for i in [2, 5, 10, 20, 30, 50]:
    prop[f'orders_{i}_pct'] = prop[f'orders_{i}'] / prop['orders']


## calculate difference

In [67]:
for c in [f'orders_{i}_pct' for i in [2, 5, 10, 20, 30, 50]]:
    prop[c+'_diff'] = prop.apply(lambda x: \
                show_control_diff(x, prop, c, diff_pct=False), axis=1)
temp = list(zip([f'orders_{i}_pct' for i in [2, 5, 10, 20, 30, 50]], \
                [c+'_diff' for c in [f'orders_{i}_pct' for i in [2, 5, 10, 20, 30, 50]]]))
temp_cols = [l1 for l2 in temp for l1 in l2]
prop[['month', 'bucket', 'dusers', 'orders'] + temp_cols]

,month,bucket,dusers,orders,orders_2_pct,orders_2_pct_diff,orders_5_pct,orders_5_pct_diff,orders_10_pct,orders_10_pct_diff,orders_20_pct,orders_20_pct_diff,orders_30_pct,orders_30_pct_diff,orders_50_pct,orders_50_pct_diff
9,2021-04-01,ANDROID_CONTROL,6328013,965417.0,0.092450,NaN,0.572458,NaN,0.753770,NaN,0.892683,NaN,0.953667,NaN,0.986860,NaN
15,2021-04-01,ANDROID_SHOW,120644425,18512530.0,0.089486,-0.002964,0.573945,0.001487,0.755384,0.001615,0.893697,0.001015,0.954226,0.000559,0.987115,0.000256
2,2021-04-01,IOS_CONTROL,3565405,451310.0,0.087024,NaN,0.558747,NaN,0.746332,NaN,0.889065,NaN,0.951634,NaN,0.986322,NaN
7,2021-04-01,IOS_SHOW,67405117,8611742.0,0.085621,-0.001404,0.564274,0.005527,0.749782,0.003450,0.890151,0.001086,0.951956,0.000322,0.986679,0.000357
4,2021-05-01,ANDROID_CONTROL,6314464,872188.0,0.096243,NaN,0.567758,NaN,0.754763,NaN,0.895090,NaN,0.955594,NaN,0.987594,NaN
8,2021-05-01,ANDROID_SHOW,120668163,16809959.0,0.091696,-0.004547,0.567737,-0.000021,0.754788,0.000025,0.894645,-0.000445,0.955267,-0.000327,0.987633,0.000038
1,2021-05-01,IOS_CONTROL,3521527,421995.0,0.091944,NaN,0.553080,NaN,0.744452,NaN,0.888098,NaN,0.951653,NaN,0.986455,NaN
0,2021-05-01,IOS_SHOW,66655314,8055011.0,0.089219,-0.002725,0.556148,0.003068,0.746918,0.002466,0.889351,0.001253,0.951890,0.000237,0.986688,0.000233
14,2021-06-01,ANDROID_CONTROL,5366595,697933.0,0.100088,NaN,0.578951,NaN,0.764909,NaN,0.900903,NaN,0.958129,NaN,0.988092,NaN
13,2021-06-01,ANDROID_SHOW,102923157,13550539.0,0.095026,-0.005063,0.581009,0.002058,0.766902,0.001994,0.901642,0.000739,0.958158,0.000029,0.988013,-0.000079


## proportion test

In [62]:
from statsmodels.stats.proportion import proportions_ztest
# https://www.statsmodels.org/devel/generated/statsmodels.stats.proportion.proportions_ztest.html

def show_control_prop_test(x, data, pct_col, nobs_col, is_count=False, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    pct_col: column name of numerator or the percentage calcualted
    nobs_col: column name of denominator
    is_count: False if pct_col is percentage, True if pct_col is the integer (numerator)
    """
    if 'CONTROL' in x['bucket']:
        return
    if 'SHOW' in x['bucket']:
        n_control = data[(data['month']==x['month']) & \
                        (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][nobs_col].iloc[0]
        n_show = x[nobs_col]
        if is_count: 
            count_control = data[(data['month']==x['month']) & \
                                 (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][pct_col].iloc[0]
            count_show = x[pct_col]
        else:
            count_control = int(round(data[(data['month']==x['month']) & \
                                     (data['bucket'].apply(lambda p: p == x['bucket'].split('_')[0]+'_CONTROL'))][pct_col].iloc[0] * n_control,0))
            count_show = int(round(x[pct_col] * n_show, 0))
        
        stat, pval = proportions_ztest((count_control,count_show), (n_control,n_show), alternative=alternative)
        
        return round(pval, 2)

In [68]:
for c in [f'orders_{i}_pct' for i in [2, 5, 10, 20, 30, 50]]:
    prop[c+'_pval'] = prop.apply(lambda x: \
         show_control_prop_test(x, prop, c, 'dusers', is_count=False, alternative='two-sided')
         , axis=1)
    prop[c+'_diff'] = prop.apply(lambda x: f"{round(x[c+'_diff']*100, 2)}" + \
                             ("" if np.isnan(x[c+'_diff']) else "%") + \
                             ('***' if x[c+'_pval']<=0.05 else ''), axis=1)
    

temp = zip([f'orders_{i}_pct' for i in [2, 5, 10, 20, 30, 50]],\
            [c+'_diff' for c in [f'orders_{i}_pct' for i in [2, 5, 10, 20, 30, 50]]])#, [c+'_test' for c in prop_cols])
temp_cols = [l1 for l2 in temp for l1 in l2]

prop[['month', 'bucket', 'dusers', 'orders']+temp_cols]#.to_clipboard()

,month,bucket,dusers,orders,orders_2_pct,orders_2_pct_diff,orders_5_pct,orders_5_pct_diff,orders_10_pct,orders_10_pct_diff,orders_20_pct,orders_20_pct_diff,orders_30_pct,orders_30_pct_diff,orders_50_pct,orders_50_pct_diff
9,2021-04-01,ANDROID_CONTROL,6328013,965417.0,0.092450,nan,0.572458,nan,0.753770,nan,0.892683,nan,0.953667,nan,0.986860,nan
15,2021-04-01,ANDROID_SHOW,120644425,18512530.0,0.089486,-0.3%***,0.573945,0.15%***,0.755384,0.16%***,0.893697,0.1%***,0.954226,0.06%***,0.987115,0.03%***
2,2021-04-01,IOS_CONTROL,3565405,451310.0,0.087024,nan,0.558747,nan,0.746332,nan,0.889065,nan,0.951634,nan,0.986322,nan
7,2021-04-01,IOS_SHOW,67405117,8611742.0,0.085621,-0.14%***,0.564274,0.55%***,0.749782,0.35%***,0.890151,0.11%***,0.951956,0.03%***,0.986679,0.04%***
4,2021-05-01,ANDROID_CONTROL,6314464,872188.0,0.096243,nan,0.567758,nan,0.754763,nan,0.895090,nan,0.955594,nan,0.987594,nan
8,2021-05-01,ANDROID_SHOW,120668163,16809959.0,0.091696,-0.45%***,0.567737,-0.0%,0.754788,0.0%,0.894645,-0.04%***,0.955267,-0.03%***,0.987633,0.0%
1,2021-05-01,IOS_CONTROL,3521527,421995.0,0.091944,nan,0.553080,nan,0.744452,nan,0.888098,nan,0.951653,nan,0.986455,nan
0,2021-05-01,IOS_SHOW,66655314,8055011.0,0.089219,-0.27%***,0.556148,0.31%***,0.746918,0.25%***,0.889351,0.13%***,0.951890,0.02%***,0.986688,0.02%***
14,2021-06-01,ANDROID_CONTROL,5366595,697933.0,0.100088,nan,0.578951,nan,0.764909,nan,0.900903,nan,0.958129,nan,0.988092,nan
13,2021-06-01,ANDROID_SHOW,102923157,13550539.0,0.095026,-0.51%***,0.581009,0.21%***,0.766902,0.2%***,0.901642,0.07%***,0.958158,0.0%,0.988013,-0.01%


In [69]:
prop[['month', 'bucket', 'dusers', 'orders']+temp_cols].reset_index().style.applymap(color_pval)

,index,month,bucket,dusers,orders,orders_2_pct,orders_2_pct_diff,orders_5_pct,orders_5_pct_diff,orders_10_pct,orders_10_pct_diff,orders_20_pct,orders_20_pct_diff,orders_30_pct,orders_30_pct_diff,orders_50_pct,orders_50_pct_diff
0,9,2021-04-01 00:00:00,ANDROID_CONTROL,6328013,965417.000000,0.092450,nan,0.572458,nan,0.753770,nan,0.892683,nan,0.953667,nan,0.986860,nan
1,15,2021-04-01 00:00:00,ANDROID_SHOW,120644425,18512530.000000,0.089486,-0.3%***,0.573945,0.15%***,0.755384,0.16%***,0.893697,0.1%***,0.954226,0.06%***,0.987115,0.03%***
2,2,2021-04-01 00:00:00,IOS_CONTROL,3565405,451310.000000,0.087024,nan,0.558747,nan,0.746332,nan,0.889065,nan,0.951634,nan,0.986322,nan
3,7,2021-04-01 00:00:00,IOS_SHOW,67405117,8611742.000000,0.085621,-0.14%***,0.564274,0.55%***,0.749782,0.35%***,0.890151,0.11%***,0.951956,0.03%***,0.986679,0.04%***
4,4,2021-05-01 00:00:00,ANDROID_CONTROL,6314464,872188.000000,0.096243,nan,0.567758,nan,0.754763,nan,0.895090,nan,0.955594,nan,0.987594,nan
5,8,2021-05-01 00:00:00,ANDROID_SHOW,120668163,16809959.000000,0.091696,-0.45%***,0.567737,-0.0%,0.754788,0.0%,0.894645,-0.04%***,0.955267,-0.03%***,0.987633,0.0%
6,1,2021-05-01 00:00:00,IOS_CONTROL,3521527,421995.000000,0.091944,nan,0.553080,nan,0.744452,nan,0.888098,nan,0.951653,nan,0.986455,nan
7,0,2021-05-01 00:00:00,IOS_SHOW,66655314,8055011.000000,0.089219,-0.27%***,0.556148,0.31%***,0.746918,0.25%***,0.889351,0.13%***,0.951890,0.02%***,0.986688,0.02%***
8,14,2021-06-01 00:00:00,ANDROID_CONTROL,5366595,697933.000000,0.100088,nan,0.578951,nan,0.764909,nan,0.900903,nan,0.958129,nan,0.988092,nan
9,13,2021-06-01 00:00:00,ANDROID_SHOW,102923157,13550539.000000,0.095026,-0.51%***,0.581009,0.21%***,0.766902,0.2%***,0.901642,0.07%***,0.958158,0.0%,0.988013,-0.01%
